In [1]:
import pandas as pd
path = "C:/Users/shrra/Downloads/2024-06-25T13-17_export.csv"
datafram =pd.read_csv(path)

In [2]:
def determine_new_race_5(row):
    if row['race___5'] and row['ethnicity'] == "Hispanic or Latino":
        return False
    else:
        return row['race___5']

# Apply the function to create the new column
datafram['new_race___5'] = datafram.apply(determine_new_race_5, axis=1)

# Remove rows where race___5 is false and race___8 is true
datafram = datafram[~((datafram['race___5'] == False) & (datafram['race___8'] == True))]

# race_columns = ['race___1', 'race___2', 'race___3', 'race___4', 'race___5','race___6', 'race___7','race___8','new_race___5']  
# race_counts = datafram[race_columns].sum()
# plt.figure(figsize=(10, 6))
# sns.barplot(x=race_counts.index, y=race_counts.values, palette='viridis')
# plt.xlabel('Race')
# plt.ylabel('Count')
# plt.title('Number of Counts Available for Each Race')
# plt.show()

In [7]:
import os
import json
df = datafram


def add_json_contents_to_df(df, base_folder_path):
    # Initialize a list to hold dictionaries of JSON data
    json_data_list = []
    
    for record_id in df['record_id']:
        json_file_name = f"sub-{record_id}_participant.json"
        folder_path = os.path.join(base_folder_path, f'sub-{record_id}')
        json_file_path = os.path.join(folder_path, json_file_name)
        
        if os.path.exists(json_file_path):
            with open(json_file_path, 'r') as file:
                json_data = json.load(file)
                emp_data = {'record_id': record_id}
                
                for item in json_data.get('item', []):
                    link_id = item.get('linkId')
                    answer_value = None
                    
                    # Check if 'answer' key exists and is not empty
                    if 'answer' in item and item['answer']:
                        answer = item['answer'][0]
                        if 'valueString' in answer:
                            answer_value = answer['valueString']
                        elif 'valueBoolean' in answer:
                            answer_value = answer['valueBoolean']
                    
                    emp_data[link_id] = answer_value
                
                json_data_list.append(emp_data)
        else:
            print(f"JSON file not found for record_id: {record_id}")
    
    # Create a DataFrame from the JSON data list
    json_df = pd.DataFrame(json_data_list)
    
    # Merge the original df with the new json_df on 'record_id'
    merged_df = pd.merge(df, json_df, on='record_id', how='left')
    
    return merged_df
 

# Assuming 'df' is your original DataFrame
# Replace 'base_folder_path' with your actual folder path
merged_df = add_json_contents_to_df(df, base_folder_path='C:/Users/shrra/Downloads/Bridge2AI/bridge2ai-voice-corpus-2-including-sensitive-recordings1/bids_with_sensitive_recordings/')
print(merged_df)


     Unnamed: 0                             record_id  \
0             0  8d5dc52b-e8aa-42e7-ae54-8f05c4667d39   
1             1  1b07b18b-26f9-405b-a466-29442306a7fe   
2             2  e5db3e0c-6589-4a15-a5e7-8a95e4ed34a5   
3             3  943a8bbc-bba0-4853-825c-10cae1b26ddd   
4             4  d5aeba9f-c910-4b65-81e8-0e9ad15097e7   
..          ...                                   ...   
174         174  2b1c662a-59e0-4b69-9161-a2fd4636890b   
175         175  66a03988-159c-4a3a-954b-7b4fc8320b51   
176         176  dbd7641f-f3db-4d08-8f63-2e0e7bbb8f3d   
177         177  8e9349f8-64f6-4558-ac9b-4d15c7649a80   
178         178  ba749edd-38d9-4f87-8742-c71010177050   

                  demographics_session_id  demographics_duration  \
0    B176636C-3330-4AB4-93A9-1E2305506407                    173   
1    8F8E68BB-E68C-4EA5-B71A-17D7AAE915C2                    258   
2    B94FE4BC-79FF-46A1-86CC-628E2D77874E                    249   
3    35002E1F-A963-4E9C-8F2A-738839D83C4D  

In [ ]:
merged_df.to_csv('C:/Users/shrra/Downloads/merged_try.csv')

In [ ]:
selected_df = merged_df
selected_columns = ['record_id', 'demographics_session_id', 'new_race___5','asthma']
df = selected_df[selected_columns]

In [ ]:
from scipy.stats import chi2_contingency
import statsmodels.api as sm
contingency_table = pd.crosstab(selected_df['new_race___5'], selected_df['asthma'])

# Perform chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-squared test statistic: {chi2}")
print(f"p-value: {p}")

if p < 0.05:
    print("There is a significant association between new_race___5 and asthma.")
else:
    print("There is no significant association between new_race___5 and asthma.")

In [ ]:
# df = pd.DataFrame()

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import wave


df = selected_df[selected_columns]

# Add 'subject_id' column
df['subject_id'] = 'sub-' + df['record_id'] + '/ses-' + df['demographics_session_id']

# Function to check file existence and calculate lengths
def get_valid_subject_ids_and_lengths(df, base_dir='C:/Users/shrra/Downloads/Bridge2AI/bridge2ai-voice-corpus-2-including-sensitive-recordings1/bids_with_sensitive_recordings/'):
    valid_ids = []
    all_lengths = []
    for subject_id in df['subject_id']:
        session_dir = os.path.join(base_dir, subject_id, 'audio')
        if os.path.exists(session_dir):
            for file_name in os.listdir(session_dir):
                if file_name.endswith('Rainbow-Passage_rec-Rainbow-Passage.wav'):  # Check for .wav files
                    file_path = os.path.join(session_dir, file_name)
                    with wave.open(file_path, 'r') as wav_file:
                        frames = wav_file.getnframes()
                        rate = wav_file.getframerate()
                        duration = frames / float(rate)
                    valid_ids.append(subject_id)
                    all_lengths.append(duration)
                    break
    return valid_ids, all_lengths

# Get valid subject IDs and lengths
valid_subject_ids, all_lengths = get_valid_subject_ids_and_lengths(df)

# Filter the dataframe to include only records with valid subject IDs
df = df[df['subject_id'].isin(valid_subject_ids)]

# Split the data into train+val and test sets (80% train+val, 20% test)
train_val_df, test_df = train_test_split(df, test_size=0.2, stratify=df['asthma'], random_state=42)

# Split the train+val set into train and validation sets (80% train, 20% validation)
train_df, val_df = train_test_split(train_val_df, test_size=0.25, stratify=train_val_df['asthma'], random_state=42)  # 0.25 * 0.8 = 0.2

# Display the number of records in each distribution
def display_distribution(df, name):
    print(f"Distribution in {name}:")
    print(df['asthma'].value_counts())
    print()
# display_distribution(selected_df, 'OG Set')
display_distribution(df, 'Filtered Set')
display_distribution(train_df, 'Train Set')
display_distribution(val_df, 'Validation Set')
display_distribution(test_df, 'Test Set')

# Calculate the overall minimum and maximum lengths
overall_min_length = min(all_lengths)
overall_max_length = max(all_lengths)

# Display the overall min and max lengths
print(f"Overall Minimum Length of .wav files: {overall_min_length:.2f} seconds")
print(f"Overall Maximum Length of .wav files: {overall_max_length:.2f} seconds")


In [ ]:
import numpy as np
import librosa
import os
import librosa.display
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Define constants
AUDIO_LENGTH = 30  # in seconds
SAMPLE_RATE = 16000  # in Hz

# Function to convert audio to spectrogram
def audio_to_spectrogram(file_path):
    y, sr = librosa.load(file_path, sr=SAMPLE_RATE, duration=AUDIO_LENGTH)
    spect = librosa.feature.melspectrogram(y=y, sr=sr)
    spect_db = librosa.power_to_db(spect, ref=np.max)
    return spect_db

# Function to pad or truncate spectrogram to a fixed size
def pad_or_truncate_spectrogram(spectrogram, fixed_shape):
    if spectrogram.shape[1] > fixed_shape[1]:
        spectrogram = spectrogram[:, :fixed_shape[1]]
    else:
        pad_width = fixed_shape[1] - spectrogram.shape[1]
        spectrogram = np.pad(spectrogram, ((0, 0), (0, pad_width)), mode='constant')
    return spectrogram

# Process dataset
def process_dataset(df, fixed_shape=None, base_dir='C:/Users/shrra/Downloads/Bridge2AI/bridge2ai-voice-corpus-2-including-sensitive-recordings1/bids_with_sensitive_recordings/'):
    X = []
    y = []
    max_shape = (0, 0)
    
    # First pass to determine the maximum shape if fixed_shape is not provided
    if fixed_shape is None:
        for idx, row in df.iterrows():
            subject_id = row['subject_id']
            session_dir = os.path.join(base_dir, subject_id, 'audio')
            for file_name in os.listdir(session_dir):
                if file_name.endswith('Rainbow-Passage_rec-Rainbow-Passage.wav'):
                    file_path = os.path.join(session_dir, file_name)
                    spectrogram = audio_to_spectrogram(file_path)
                    max_shape = (max(max_shape[0], spectrogram.shape[0]), max(max_shape[1], spectrogram.shape[1]))
                    break
        fixed_shape = max_shape

    # Second pass to process the data
    for idx, row in df.iterrows():
        subject_id = row['subject_id']
        session_dir = os.path.join(base_dir, subject_id, 'audio')
        for file_name in os.listdir(session_dir):
            if file_name.endswith('Rainbow-Passage_rec-Rainbow-Passage.wav'):
                file_path = os.path.join(session_dir, file_name)
                spectrogram = audio_to_spectrogram(file_path)
                spectrogram = pad_or_truncate_spectrogram(spectrogram, fixed_shape)
                X.append(spectrogram)
                y.append(row['asthma'])
                break
    return np.array(X), np.array(y), fixed_shape

# Prepare data
X_train, y_train, fixed_shape = process_dataset(train_df)
X_val, y_val, _ = process_dataset(val_df, fixed_shape=fixed_shape)
X_test, y_test, _ = process_dataset(test_df, fixed_shape=fixed_shape)

# Encode labels
label_encoder = LabelEncoder()
y_train = to_categorical(label_encoder.fit_transform(y_train))
y_val = to_categorical(label_encoder.transform(y_val))
y_test = to_categorical(label_encoder.transform(y_test))

# Ensure inputs have the right shape
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]

# Build CNN model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(fixed_shape[0], fixed_shape[1], 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(np.unique(y_train)), activation='softmax')
])

# Compile model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

# Evaluate model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
conf_matrix = confusion_matrix(y_true, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=label_encoder.classes_)

# Display confusion matrix
disp.plot(cmap=plt.cm.Blues)
plt.show()


# Display training history
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
